In [1]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!bash Anaconda3-5.1.0-Linux-x86_64.sh
!source ~/.bashrc
!conda install pytorch=0.4.1

--2023-11-04 04:58:51--  https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M  81.0MB/s    in 8.6s    

2023-11-04 04:59:00 (64.3 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ saved [577996269/577996269]


Welcome to Anaconda3 5.1.0

In order to continue the installation process, please review the license
agreement.
Please, press ENTER to continue
>>> 
Anaconda End User License Agreement

Copyright 2015, Anaconda, Inc.

All rights reserved under the 3-clause BSD License:

Redistribution and use in source and binary forms, with or without modification, are permitted provi
ded that the following condition

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
%cd /content/drive/MyDrive/Robust_Adversarial_Learning

/content/drive/MyDrive/Robust_Adversarial_Learning


In [4]:
from __future__ import print_function
import os
import argparse
import torch.nn as nn
import torchvision
import numpy as np
import torch.nn.functional as F
from losses.trades import trades_loss
import copy
import torch
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from models.wideresnet import *
from models.resnet import *
from models.small_cnn import *
from torch.utils.data import Dataset, DataLoader

In [5]:
# Use more of these as complexity increases

class Data:
  def __init__(self, train_loader, test_loader, attack_loader):
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.attack_loader = attack_loader

class Model:
  model = None
  def __init__(self, id):
    self.id = id

class Loss:
  def __init__(self, loss_fn, id=None):
    self.loss_fn = loss_fn
    self.id = id

class Configuration:
  def __init__(self, data, model, loss, attack, model_pt=None, id=None):
    self.data = data
    self.model = model
    self.loss = loss
    self.attack = attack
    self.model_pt = model_pt # Should move this to model

    self.id = id

  def getConfig(self):
    return self.data, self.model, self.loss, self.attack

  def getId(self):
    return self.id

In [6]:
configurations = []

In [7]:
use_cuda = torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
transform_attack = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
cifar10_train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, **kwargs)
testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
cifar10_test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, **kwargs)
attackset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_attack)
cifar10_attack_loader = torch.utils.data.DataLoader(attackset, batch_size=200, shuffle=False, **kwargs)

cifar10_data = Data(cifar10_train_loader, cifar10_test_loader, cifar10_attack_loader)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [16]:
class CIFAR10CDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
%pwd
images = np.load('../data/CIFAR-10-C/spatter.npy') # Set this to whatever we want
labels = np.load('../data/CIFAR-10-C/labels.npy')

# Set 200 to test the whole thing
images_severity_1 = images[:200]
images_severity_5 = images[-200:]
labels_severity_1 = labels[:200]
labels_severity_5 = labels[-200:]

cifar10c_dataset = CIFAR10CDataset(
    data=np.concatenate((images_severity_1, images_severity_5)),
    labels=np.concatenate((labels_severity_1, labels_severity_5)),
    transform=transform
)

In [17]:
cifar10c_attack_loader = DataLoader(cifar10c_dataset, batch_size=200, shuffle=False)
cifar10_c_data = Data(cifar10_train_loader, cifar10_test_loader, cifar10c_attack_loader)

In [18]:
def identity_attack(model, X, y):
  out = model(X)
  acc = (out.data.max(1)[1] == y.data).float().sum()
  return acc.item()

def pgd_attack(epsilon=0.031, num_steps=20, step_size=0.003):

  def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=epsilon,
                  num_steps=num_steps,
                  step_size=step_size):
    # Below two for natural accuracy
    # out = model(X)
    # err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    # if args.random:
    #     random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(device)
    #     X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    acc_pgd = (model(X_pgd).data.max(1)[1] == y.data).float().sum()
    print('err pgd (white-box): ', acc_pgd)
    return acc_pgd.item()

  return _pgd_whitebox

In [19]:
### Add Configurations

def general_trades_loss_fn(beta=6.0, epsilon=0.3, step_size=0.007, num_steps=10):
  def trades_loss_fn(model, data, target, optimizer):
    return trades_loss(model=model, x_natural=data, y=target, optimizer=optimizer, step_size=step_size,
                      epsilon=epsilon, perturb_steps=num_steps, beta=beta, distance='l_inf')
  return trades_loss_fn

for beta in [0.6]:
  id = f'CIFARC10:RESNET18:TRADES_LOSS:BETA={beta}'
  model_pt = os.path.join("weights", f'CIFARC10:RESNET18:TRADES_LOSS:BETA={beta}_ep=0.pt')
  resnet18 = ResNet18().to(device)
  trades_loss_beta = Loss(general_trades_loss_fn(beta=beta), id)
  config1 = Configuration(cifar10_c_data, resnet18, trades_loss_beta, identity_attack, model_pt, trades_loss_beta.id)
  configurations.append(config1)

# Ideally, all we have to do is just add our loss with same configs and run the whole thing

In [20]:
training_loss = {}
natural_accuracy = {}
robustness_accuracy = {}

In [21]:
def train(model, data_loader, loss, config, epochs=1):

  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
  # TODO: We can move the optimizer to a field of Loss object

  model.to(device)
  if config.model_pt is not None:
    model.load_state_dict(torch.load(config.model_pt))

  for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_idx, (data, target) in enumerate(data_loader):
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          l = loss.loss_fn(model, data, target, optimizer)
          l.backward()
          optimizer.step()
          total_loss += l.item()

          print(loss.id + " @ Batch idx " + str(batch_idx) + " / " + str(len(data_loader) - 1))

    torch.save(model.state_dict(), os.path.join("weights", loss.id + "_ep=" + str(epoch) + ".pt"))
    torch.save(optimizer.state_dict(), os.path.join("optimizers", loss.id + "_ep=" + str(epoch) + ".tar"))

  return total_loss

def accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return 100. * correct / total

def robust_accuracy(model, attack, data_loader):

    model.eval()
    correct = 0
    total = 0

    for data, target in data_loader:
        data, target = data.to(device), target.to(device)

        X, y = Variable(data, requires_grad=True), Variable(target)
        correct_count = attack(model, X, y)
        correct += correct_count
        total += target.size(0)
    return 100. * correct / total

In [22]:
for (c, configuration) in enumerate(configurations):
  data, model, loss, attack = configuration.getConfig()

  training_loss[configuration.getId()] = train(model, data.train_loader, loss, configuration)
  natural_accuracy[configuration.getId()] = accuracy(model, data.test_loader)
  robustness_accuracy[configuration.getId()] = robust_accuracy(model, attack, data.attack_loader)

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


CIFARC10:RESNET18:TRADES_LOSS:BETA=0.6 @ Batch idx 0 / 390


In [23]:
print(training_loss)
print(natural_accuracy)
print(robustness_accuracy)

# TODO: Save this stuff to file

{'CIFARC10:RESNET18:TRADES_LOSS:BETA=0.6': 2.2139861583709717}
{'CIFARC10:RESNET18:TRADES_LOSS:BETA=0.6': 9.97}
{'CIFARC10:RESNET18:TRADES_LOSS:BETA=0.6': 7.25}
